## PURPOSE OF THIS NOTEBOOK

This notebook is dedicated to the extraction, selection, and light harmonisation of 
non-financial, non-signal company features that will be used later for modelling and
prospectability scoring.

At this stage, the objective is **not** to perform imputations, 
apply modelling logic or compute scores. 

Instead, we focus on:
- selecting only relevant columns from each dataset,
- ensuring consistent identifiers (`siren`),
- merging compatible datasets into a single, clean table,
- inspecting data quality (missing values, data types),
- producing a reusable feature table for downstream notebooks.

The resulting dataset is designed to be joined later with:
- financial and KPI features,
- aggregated signal features,
- territorial indicators (INSEE),
- company age extracted from legacy JSON files.
These are all processed in separate notebooks.

All heavy transformations are intentionally deferred to downstream, model-specific notebooks
to preserve clarity, modularity, and reproducibility of the pipeline.


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style
# sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Set path to datasets
DATA_DIR = Path("/home/tella/code/StellaRodriguesLallement/OSE_Project/Dataset_visualization/new_data_folder/data/extracted_datasets")

# Check it exists
print(DATA_DIR.exists())

print("Libraries imported successfully!")

True
Libraries imported successfully!


In [2]:
# LOADING DATASETS

df_basic     = pd.read_csv(DATA_DIR / '01_company_basic_info.csv', dtype={'siren': str})
df_workforce = pd.read_csv(DATA_DIR / '03_workforce_data.csv', dtype={'siren': str})
df_structure = pd.read_csv(DATA_DIR / '04_company_structure.csv', dtype={'siren': str})
df_flags     = pd.read_csv(DATA_DIR / '05_classification_flags.csv', dtype={'siren': str})
# df_contacts  = pd.read_csv(DATA_DIR / '06_contact_metrics.csv', dtype={'siren': str})        # not used
# df_financial = pd.read_csv(DATA_DIR / '02_financial_data.csv', dtype={'siren': str})         # treated in another notebook
# df_kpi       = pd.read_csv(DATA_DIR / '07_kpi_data.csv', dtype={'siren': str})               # treated in another notebook
# df_signals   = pd.read_csv(DATA_DIR / '08_signals.csv', dtype={'siren': str})                # treated in another notebook
# df_articles  = pd.read_csv(DATA_DIR / '09_articles.csv', dtype={'siren': str})               # not used

print("All relevant datasets loaded successfully!")


All relevant datasets loaded successfully!


## BASIC FEATURES

In [3]:
df_basic.columns

Index(['company_name', 'siren', 'siret', 'address', 'cp', 'departement',
       'departement_id', 'juridic_form', 'last_modified', 'naf_code',
       'naf_label', 'processedAt', 'raison_sociale', 'raison_sociale_keyword',
       'resume_activite', 'updatedAt', 'ville'],
      dtype='object')

In [4]:
df_basic.shape

(18116, 17)

In [5]:
df_basic.head(3)

,company_name,siren,siret,address,cp,departement,departement_id,juridic_form,last_modified,naf_code,naf_label,processedAt,raison_sociale,raison_sociale_keyword,resume_activite,updatedAt,ville
0,AVI-CHARENTE,305689432,3.056894e+13,9 rue Galilee ZAC DE BELLE AIRE 17440 AYTRE,17440,NaN,NaN,NaN,2025-09-13T09:38:58+02:00,NaN,NaN,2014-10-13T08:16:59+02:00,AVI-CHARENTE,NaN,L'entreprise se spécialise dans la fabrication...,2025-09-13T09:38:58+02:00,AYTRE
1,SOCIETE D'ABATTAGE DE MONTMORILLON,752129643,7.521296e+13,rue Pierre Pagenaud ZI SUD 86500 MONTMORILLON,86500,NaN,NaN,NaN,2025-09-13T09:19:15+02:00,NaN,NaN,2015-05-04T09:32:37+02:00,SOCIETE D'ABATTAGE DE MONTMORILLON,ABATTOIR DE MONTMORILLON - 86500 - MONTMORILLON,Cette entreprise se consacre à l'élevage et à ...,2025-09-13T09:19:15+02:00,MONTMORILLON
2,COVI,391892171,3.918922e+13,boulevard du Marechal Foch 79300 BRESSUIRE,79300,NaN,NaN,NaN,2025-09-13T06:55:55+02:00,NaN,NaN,2013-09-12T14:20:49+02:00,COVI,COVI - 79300 - BRESSUIRE,"Fabricant de plats cuisinés, conserves de vian...",2025-09-13T06:55:55+02:00,BRESSUIRE


In [6]:
df_basic.isnull().mean().sort_values(ascending=False)

departement               1.000000
departement_id            1.000000
juridic_form              1.000000
naf_code                  1.000000
naf_label                 1.000000
raison_sociale_keyword    0.894624
siret                     0.025723
cp                        0.012089
ville                     0.011978
raison_sociale            0.000000
updatedAt                 0.000000
resume_activite           0.000000
company_name              0.000000
processedAt               0.000000
siren                     0.000000
address                   0.000000
last_modified             0.000000
dtype: float64

In [7]:
df_basic_feat = df_basic[['company_name','siren','siret','ville']].copy()
df_basic_feat.head(3)

,company_name,siren,siret,ville
0,AVI-CHARENTE,305689432,3.056894e+13,AYTRE
1,SOCIETE D'ABATTAGE DE MONTMORILLON,752129643,7.521296e+13,MONTMORILLON
2,COVI,391892171,3.918922e+13,BRESSUIRE


In [8]:
df_basic_feat['siren'].value_counts().sort_values(ascending=False).head(12)

siren
383907425    2
934180183    2
813542461    2
398553586    2
818676264    2
396580102    2
953565785    2
530058288    2
534948617    2
915121883    2
437485758    2
380678250    2
Name: count, dtype: int64

In [9]:
df_basic_feat = df_basic_feat.drop_duplicates()

In [10]:
df_basic_feat.shape

(18050, 4)

In [11]:
df_basic_feat['siren'].value_counts().sort_values(ascending=False).head(3)

siren
891871196    2
957802416    1
451972483    1
Name: count, dtype: int64

In [12]:
df_basic_feat[df_basic_feat['siren'] == '891871196']


,company_name,siren,siret,ville
102,GFA DU MOULIN DE L'ETANG,891871196,8.918712e+13,GOURGE
16994,DU MOULIN DE L'ETANG,891871196,8.918712e+13,GOURGE


In [13]:
df_basic_feat = (
    df_basic_feat
        .sort_values(by='company_name', key=lambda s: s.str.len(),ascending=False)
        .drop_duplicates(subset='siren', keep='first')
)

In [14]:
df_basic_feat['siren'].nunique(), len(df_basic_feat)

(18049, 18049)

In [15]:
df_basic_feat.shape

(18049, 4)

## WORKFORCE FEATURES

In [16]:
df_workforce.columns

Index(['company_name', 'siren', 'siret', 'effectif', 'effectifConsolide',
       'effectifGroupe', 'trancheEffectifConsolide', 'trancheEffectifPrecis'],
      dtype='object')

In [17]:
df_workforce.shape

(18116, 8)

In [18]:
df_workforce.head(3)

,company_name,siren,siret,effectif,effectifConsolide,effectifGroupe,trancheEffectifConsolide,trancheEffectifPrecis
0,AVI-CHARENTE,305689432,3.056894e+13,225.0,NaN,NaN,NaN,27.0
1,SOCIETE D'ABATTAGE DE MONTMORILLON,752129643,7.521296e+13,35.0,NaN,NaN,NaN,25.0
2,COVI,391892171,3.918922e+13,225.0,NaN,NaN,NaN,27.0


In [19]:
df_workforce.isnull().mean().sort_values(ascending=False)

effectifConsolide           0.979245
effectifGroupe              0.979245
trancheEffectifConsolide    0.976926
effectif                    0.322588
trancheEffectifPrecis       0.297858
siret                       0.025723
company_name                0.000000
siren                       0.000000
dtype: float64

In [20]:
mask_effectif_missing = df_workforce['effectif'].isna()
df_workforce.loc[mask_effectif_missing, 'trancheEffectifPrecis'].notna().value_counts()

trancheEffectifPrecis
False    5391
True      453
Name: count, dtype: int64

### Why keep `trancheEffectifPrecis`?
Many rows with missing `effectif` still have a valid `trancheEffectifPrecis`.  
This makes the tranche column useful later as an imputation helper (e.g. using tranche midpoints).  
No imputation is done here; the variable is simply preserved.


In [21]:
df_workforce_feat = df_workforce[['siren','effectif','trancheEffectifPrecis']].copy()

In [22]:
df_workforce_feat['siren'].value_counts().sort_values(ascending=False).head(12)

siren
383907425    2
934180183    2
813542461    2
398553586    2
818676264    2
396580102    2
953565785    2
530058288    2
534948617    2
915121883    2
437485758    2
380678250    2
Name: count, dtype: int64

In [23]:
df_workforce_feat = df_workforce_feat.drop_duplicates()

In [24]:
df_workforce_feat.shape

(18057, 3)

In [25]:
df_workforce_feat['siren'].value_counts().sort_values(ascending=False).head(12)

siren
891871196    2
395169576    2
615850104    2
484585195    2
000240696    2
951566710    2
915121883    2
920717550    2
313944134    1
478608037    1
560500498    1
322304197    1
Name: count, dtype: int64

In [26]:
df_workforce_feat[df_workforce_feat['siren'].duplicated(keep=False)].sort_values('siren')


,siren,effectif,trancheEffectifPrecis
103,000240696,0.0,NaN
1514,000240696,NaN,NaN
8058,395169576,NaN,NaN
8063,395169576,NaN,25.0
104,484585195,20.0,25.0
12060,484585195,4.0,23.0
17825,615850104,25.0,25.0
17937,615850104,NaN,NaN
102,891871196,0.0,NaN
16994,891871196,NaN,NaN


In [27]:
df_workforce_feat = df_workforce_feat.dropna(subset=['effectif', 'trancheEffectifPrecis'], how='all')


In [28]:
df_workforce_feat.shape

(12687, 3)

In [29]:
df_workforce_feat[df_workforce_feat['siren'].duplicated(keep=False)].sort_values('siren')

,siren,effectif,trancheEffectifPrecis
104,484585195,20.0,25.0
12060,484585195,4.0,23.0
100,915121883,0.0,NaN
920,915121883,1.0,23.0


In [30]:
df_workforce[df_workforce['siren'] == '484585195']

,company_name,siren,siret,effectif,effectifConsolide,effectifGroupe,trancheEffectifConsolide,trancheEffectifPrecis
104,NATURE ET BIO,484585195,4.845852e+13,20.0,0.0,0.0,NaN,25.0
12060,NATURE ET BIO,484585195,4.845852e+13,4.0,NaN,NaN,NaN,23.0


In [31]:
df_workforce[df_workforce['siren'] == '915121883']

,company_name,siren,siret,effectif,effectifConsolide,effectifGroupe,trancheEffectifConsolide,trancheEffectifPrecis
100,MAMIE MESURE LORIENT,915121883,9.151219e+13,0.0,0.0,0.0,NaN,NaN
920,MAMIE MESURE LORIENT,915121883,9.151219e+13,1.0,NaN,NaN,NaN,23.0


In [32]:
df_workforce_feat = (
    df_workforce_feat
    .sort_values(['siren', 'effectif'], ascending=[True, False])
    .drop_duplicates(subset='siren', keep='first')
)

In [33]:
df_workforce_feat.isnull().mean().sort_values(ascending=False)

effectif                 0.035711
trancheEffectifPrecis    0.000315
siren                    0.000000
dtype: float64

In [34]:
df_workforce_feat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12685 entries, 103 to 19
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   siren                  12685 non-null  object 
 1   effectif               12232 non-null  float64
 2   trancheEffectifPrecis  12681 non-null  float64
dtypes: float64(2), object(1)
memory usage: 396.4+ KB


In [35]:
df_workforce_feat.shape

(12685, 3)

## STRUCTURE FEATURES

In [36]:
df_structure.columns

Index(['company_name', 'siren', 'siret', 'groupOwnerSiren',
       'groupOwnerSocialName', 'hasEtabSecondaire', 'hasGroupOwner',
       'nbActionnaires', 'nbEtabSecondaire', 'nbMarques'],
      dtype='object')

In [37]:
df_structure.shape

(18116, 10)

In [38]:
df_structure.isnull().mean().sort_values(ascending=False)

groupOwnerSiren         0.914827
groupOwnerSocialName    0.914827
nbActionnaires          0.898543
nbMarques               0.882921
nbEtabSecondaire        0.865809
siret                   0.025723
company_name            0.000000
siren                   0.000000
hasEtabSecondaire       0.000000
hasGroupOwner           0.000000
dtype: float64

In [39]:
df_structure_feat = df_structure[['siren','nbEtabSecondaire']].copy()
df_structure_feat.head()

,siren,nbEtabSecondaire
0,305689432,NaN
1,752129643,NaN
2,391892171,4.0
3,316203942,NaN
4,451972483,2.0


In [40]:
df_structure_feat['siren'].value_counts()

siren
383907425    2
912588670    2
325423572    2
838056646    2
712051291    2
            ..
880026166    1
848906806    1
352462766    1
922256896    1
802315481    1
Name: count, Length: 18049, dtype: int64

In [41]:
df_structure_feat['siren'].nunique()

18049

In [42]:
df_structure_feat = df_structure_feat.drop_duplicates()

In [43]:
df_structure_feat.shape

(18062, 2)

In [44]:
df_structure_feat['siren'].value_counts().sort_values(ascending=False).head(15)

siren
484585195    2
380678250    2
951566710    2
534948617    2
891871196    2
437485758    2
915121883    2
000240696    2
377559265    2
339641185    2
920717550    2
395169576    2
351371257    2
380356436    1
316203942    1
Name: count, dtype: int64

In [45]:
df_structure_feat[df_structure_feat['siren'].duplicated(keep=False)].sort_values('siren')

,siren,nbEtabSecondaire
103,000240696,0.0
1514,000240696,NaN
4420,339641185,NaN
4471,339641185,4.0
11602,351371257,NaN
11601,351371257,1.0
15768,377559265,2.0
15749,377559265,NaN
8816,380678250,NaN
8817,380678250,7.0


In [46]:
df_structure_feat = (
    df_structure_feat
    .sort_values(['siren', 'nbEtabSecondaire'], ascending=[True, False])
    .drop_duplicates(subset='siren', keep='first')
)

In [47]:
df_structure_feat.shape

(18049, 2)

# FLAG FEATURES

In [48]:
df_flags.columns

Index(['company_name', 'siren', 'siret', 'cac40', 'entreprise_b2b',
       'entreprise_b2c', 'entreprise_biotech_medtech', 'entreprise_familiale',
       'fintech', 'hasBodacc', 'hasBrevets', 'hasESV1Contacts', 'hasMarques',
       'radiee', 'risk', 'site_ecommerce', 'startup'],
      dtype='object')

In [49]:
df_flags.shape

(18116, 17)

In [50]:
df_flags.isnull().mean().sort_values(ascending=False)

siret                         0.025723
company_name                  0.000000
hasBodacc                     0.000000
site_ecommerce                0.000000
risk                          0.000000
radiee                        0.000000
hasMarques                    0.000000
hasESV1Contacts               0.000000
hasBrevets                    0.000000
fintech                       0.000000
siren                         0.000000
entreprise_familiale          0.000000
entreprise_biotech_medtech    0.000000
entreprise_b2c                0.000000
entreprise_b2b                0.000000
cac40                         0.000000
startup                       0.000000
dtype: float64

In [51]:
flags_selected = [
    'siren',
    'startup',
    'site_ecommerce',
    'entreprise_b2b',
    'entreprise_b2c',
    'hasBrevets',
    'hasMarques'
]

df_flags_feat = df_flags[flags_selected].copy()
df_flags_feat.head()

,siren,startup,site_ecommerce,entreprise_b2b,entreprise_b2c,hasBrevets,hasMarques
0,305689432,False,False,True,False,False,False
1,752129643,False,False,False,False,False,False
2,391892171,False,False,True,False,False,True
3,316203942,False,True,True,False,False,False
4,451972483,False,False,True,True,False,True


In [52]:
df_flags_feat['siren'].value_counts().sort_values(ascending=False)

siren
383907425    2
934180183    2
813542461    2
398553586    2
818676264    2
            ..
000158708    1
907699458    1
538419060    1
392729935    1
802315481    1
Name: count, Length: 18049, dtype: int64

In [53]:
df_flags_feat = df_flags_feat.drop_duplicates()

In [54]:
df_flags_feat.shape

(18065, 7)

In [55]:
df_flags_feat[df_flags_feat['siren'].duplicated(keep=False)].sort_values('siren').head(7)

,siren,startup,site_ecommerce,entreprise_b2b,entreprise_b2c,hasBrevets,hasMarques
4420,339641185,False,False,False,False,False,False
4471,339641185,False,False,False,False,False,True
11602,351371257,False,False,False,False,False,False
11601,351371257,False,True,False,False,False,True
15749,377559265,False,False,False,False,False,False
15768,377559265,False,False,False,False,False,True
8817,380678250,False,False,False,False,False,True


In [56]:
# Resolve duplicate sirens by keeping the row with the most True flags

flag_cols = [
    'startup',
    'site_ecommerce',
    'entreprise_b2b',
    'entreprise_b2c',
    'hasBrevets',
    'hasMarques'
]

# 1. Count how many True values each row has
df_flags_feat['true_count'] = df_flags_feat[flag_cols].sum(axis=1)

# 2. Sort so that, for each siren, the row with the most True flags comes first
df_flags_feat = df_flags_feat.sort_values(by=['siren', 'true_count'],ascending=[True, False])

# 3. Keep one row per siren
df_flags_feat = df_flags_feat.drop_duplicates(subset='siren', keep='first')

# 4. Remove helper column
df_flags_feat = df_flags_feat.drop(columns='true_count')


In [57]:
df_flags_feat['siren'].value_counts().sort_values(ascending=False).head(7)

siren
000055901    1
000067891    1
000058864    1
000061253    1
000066152    1
000066677    1
000084981    1
Name: count, dtype: int64

In [58]:
df_flags_feat.shape

(18049, 7)

# MERGING EVERYTHING

In [59]:
df_merged = (
    df_basic_feat
    .merge(df_workforce_feat, on='siren', how='left')
    .merge(df_structure_feat, on='siren', how='left')
    .merge(df_flags_feat, on='siren', how='left')
)

In [60]:
df_merged.shape

(18049, 13)

In [61]:
df_merged.head()

,company_name,siren,siret,ville,effectif,trancheEffectifPrecis,nbEtabSecondaire,startup,site_ecommerce,entreprise_b2b,entreprise_b2c,hasBrevets,hasMarques
0,SOCIETE D ECONOMIE MIXE D AMENAGEMENT ET DE GE...,662012491,6.620125e+13,CHEVILLY-LARUE,375.0,28.0,1.0,False,True,True,True,False,True
1,BRASSERIE MISSO ; PARTAGE ET APPRENTISAGE DE L...,820959559,8.209596e+13,MARCILLY D'AZERGUES,NaN,NaN,NaN,False,False,False,False,False,False
2,COMPAGNIE DES ASSOCIES NORD EST LAIT INDUSTRIE...,412984049,4.129840e+13,PETIT-FAYT,NaN,NaN,2.0,False,False,False,False,False,False
3,LES COMPAGNONS DU MIEL - FRANCE MIEL COOPERATI...,444205421,4.442054e+13,PORT-LESNEY,NaN,NaN,1.0,False,False,True,False,False,True
4,SOCIETE COOPERATIVE ANONYME ARTISANALE D ACHAT...,693820938,6.938209e+13,MONDEVILLE,12.0,24.0,NaN,False,False,False,False,False,False


In [62]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18049 entries, 0 to 18048
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   company_name           18049 non-null  object 
 1   siren                  18049 non-null  object 
 2   siret                  17584 non-null  float64
 3   ville                  17837 non-null  object 
 4   effectif               12232 non-null  float64
 5   trancheEffectifPrecis  12681 non-null  float64
 6   nbEtabSecondaire       2431 non-null   float64
 7   startup                18049 non-null  bool   
 8   site_ecommerce         18049 non-null  bool   
 9   entreprise_b2b         18049 non-null  bool   
 10  entreprise_b2c         18049 non-null  bool   
 11  hasBrevets             18049 non-null  bool   
 12  hasMarques             18049 non-null  bool   
dtypes: bool(6), float64(4), object(3)
memory usage: 1.1+ MB


In [63]:
df_merged.isnull().mean().sort_values(ascending=False)

nbEtabSecondaire         0.865311
effectif                 0.322289
trancheEffectifPrecis    0.297413
siret                    0.025763
ville                    0.011746
company_name             0.000000
siren                    0.000000
startup                  0.000000
site_ecommerce           0.000000
entreprise_b2b           0.000000
entreprise_b2c           0.000000
hasBrevets               0.000000
hasMarques               0.000000
dtype: float64

In [64]:
df_merged.describe().T

,count,mean,std,min,25%,50%,75%,max
siret,17584.0,6.163873e+13,2.240441e+13,5.650031e+11,4.137437e+13,5.285237e+13,8.314331e+13,9.981156e+13
effectif,12232.0,6.326758e+01,6.148997e+02,0.000000e+00,4.000000e+00,1.200000e+01,3.500000e+01,6.161300e+04
trancheEffectifPrecis,12681.0,2.421347e+01,1.588035e+00,2.300000e+01,2.300000e+01,2.400000e+01,2.500000e+01,5.500000e+01
nbEtabSecondaire,2431.0,8.482929e+00,4.064278e+01,0.000000e+00,1.000000e+00,2.000000e+00,4.000000e+00,1.085000e+03


In [67]:
df_merged.to_parquet("/home/tella/code/StellaRodriguesLallement/OSE_Project/ose-main/src/ose_core/data/data_stella/merged_other_features.parquet",
    index=False)

## OUTPUT

This notebook outputs `df_merged`, a **clean, joined, non-financial feature table**
indexed by `siren`.

Characteristics:
- one row per company,
- no imputations,
- no scaling,
- no derived scores,
- missing values are preserved intentionally.

This table is designed to be joined downstream with:
- financial and KPI features,
- aggregated signal features,
- age and legal form extracted from legacy JSON sources.
